### Import Libraries

In [10]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [11]:
data = pd.read_csv("~/Personal_project/Customer-Lifetime-Value-Prediction/data/final_dataset.csv")
data.head(5)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalPrice,Recency,Frequency,Monetary,...,desc_topic_16,desc_topic_17,desc_topic_18,desc_topic_19,desc_topic_20,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour,Weekday
0,541431.0,23166.0,74215,2011-01-18 10:01:00,1.04,12346.0,77183.6,0.0,2,0.0,...,0.032051,-0.062761,0.067446,0.034499,-0.057299,2011,1,18,10,1
1,537626.0,85116.0,12,2010-12-07 14:57:00,2.10,12347.0,25.2,0.0,182,4310.0,...,-0.032339,0.023959,-0.022264,0.033915,0.013968,2010,12,7,14,1
2,537626.0,22375.0,4,2010-12-07 14:57:00,4.25,12347.0,17.0,0.0,182,4310.0,...,-0.088507,0.119232,0.051086,0.038123,-0.035789,2010,12,7,14,1
3,537626.0,71477.0,12,2010-12-07 14:57:00,3.25,12347.0,39.0,0.0,182,4310.0,...,-0.066149,-0.013725,0.000939,0.012026,0.097659,2010,12,7,14,1
4,537626.0,22492.0,36,2010-12-07 14:57:00,0.65,12347.0,23.4,0.0,182,4310.0,...,-0.052948,0.087200,0.028290,0.025431,-0.012222,2010,12,7,14,1


In [12]:
data['Quantity'].unique()

array([74215,    12,     4,    36,     6,    30,     3,    24,    10,
         240,     8,     2,    18,    16,    48,    20,    72,   120,
         144,     1,    80,    96,    25,     5,    32,    60,    40,
          15,     9,    28,   180,    64,    13,   288,   100,    50,
         256,     7,   108,    11,   272,   192,   576,   160,   168,
         200,   480,   384,   128,   216,   432,   320,   600,   400,
          84,   336,   720,  1152,   250,   960,    21,    43,    75,
          33,    42,   125,    44,    14,    70,   360,   150,    56,
         300,   912,    90,    94,    66,    17,   183,    27,    52,
         408,   224,   378,    45,    22,   102,   109,   132,   234,
         244,  1488,    19,  2040,   864,    29,    35,  1728,   291,
         462,  1200,   227,  2700,   222,   228,   246,   420,    54,
          78,  1788,  4800,   774,    41,   280,   270,   220,   350,
         348,  1900,  2880,   116,   968,   276,   700,   456,    37,
         648,   198,

### Understanding Dataset

In [13]:
print("Shape of dataset :")
print("Number of columns present : ", data.shape[1])
print("Number of rows present: ", data.shape[0])


Shape of dataset :
Number of columns present :  37
Number of rows present:  397924


In [14]:
print("Number of null values:\n", data.isnull().sum())

Number of null values:
 InvoiceNo                     0
StockCode                 34805
Quantity                      0
InvoiceDate                   0
UnitPrice                     0
CustomerID                    0
TotalPrice                    0
Recency                       0
Frequency                     0
Monetary                      0
customer_lifetime_days        0
Country_Label                 0
desc_topic_1                  0
desc_topic_2                  0
desc_topic_3                  0
desc_topic_4                  0
desc_topic_5                  0
desc_topic_6                  0
desc_topic_7                  0
desc_topic_8                  0
desc_topic_9                  0
desc_topic_10                 0
desc_topic_11                 0
desc_topic_12                 0
desc_topic_13                 0
desc_topic_14                 0
desc_topic_15                 0
desc_topic_16                 0
desc_topic_17                 0
desc_topic_18                 0
desc_topic_19   

In [15]:
data = data.dropna(subset=['StockCode'])

In [16]:
print("About data__:\n", data.describe())

About data__:
            InvoiceNo      StockCode       Quantity      UnitPrice  \
count  363119.000000  363119.000000  363119.000000  363119.000000   
mean   560820.303099   26967.327190      13.129062       2.886098   
std     13076.895436   15676.370832     188.527851       4.361971   
min    536365.000000   10002.000000       1.000000       0.000000   
25%    549547.000000   21955.000000       2.000000       1.250000   
50%    562150.000000   22603.000000       6.000000       1.700000   
75%    572237.000000   23171.000000      12.000000       3.750000   
max    581587.000000   90208.000000   80995.000000     649.500000   

          CustomerID     TotalPrice        Recency      Frequency  \
count  363119.000000  363119.000000  363119.000000  363119.000000   
mean    15295.738347      22.073616       1.337652     669.033127   
std      1711.946809     321.862986      11.934466    1464.921419   
min     12346.000000       0.000000       0.000000       1.000000   
25%     13969.0000

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363119 entries, 0 to 397923
Data columns (total 37 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   InvoiceNo               363119 non-null  float64
 1   StockCode               363119 non-null  float64
 2   Quantity                363119 non-null  int64  
 3   InvoiceDate             363119 non-null  object 
 4   UnitPrice               363119 non-null  float64
 5   CustomerID              363119 non-null  float64
 6   TotalPrice              363119 non-null  float64
 7   Recency                 363119 non-null  float64
 8   Frequency               363119 non-null  int64  
 9   Monetary                363119 non-null  float64
 10  customer_lifetime_days  363119 non-null  int64  
 11  Country_Label           363119 non-null  int64  
 12  desc_topic_1            363119 non-null  float64
 13  desc_topic_2            363119 non-null  float64
 14  desc_topic_3            3

In [18]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'CustomerID', 'TotalPrice', 'Recency', 'Frequency', 'Monetary',
       'customer_lifetime_days', 'Country_Label', 'desc_topic_1',
       'desc_topic_2', 'desc_topic_3', 'desc_topic_4', 'desc_topic_5',
       'desc_topic_6', 'desc_topic_7', 'desc_topic_8', 'desc_topic_9',
       'desc_topic_10', 'desc_topic_11', 'desc_topic_12', 'desc_topic_13',
       'desc_topic_14', 'desc_topic_15', 'desc_topic_16', 'desc_topic_17',
       'desc_topic_18', 'desc_topic_19', 'desc_topic_20', 'InvoiceYear',
       'InvoiceMonth', 'InvoiceDay', 'InvoiceHour', 'Weekday'],
      dtype='object')

In [19]:
data.drop(columns=['InvoiceDate'], inplace =True)

data.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'UnitPrice', 'CustomerID',
       'TotalPrice', 'Recency', 'Frequency', 'Monetary',
       'customer_lifetime_days', 'Country_Label', 'desc_topic_1',
       'desc_topic_2', 'desc_topic_3', 'desc_topic_4', 'desc_topic_5',
       'desc_topic_6', 'desc_topic_7', 'desc_topic_8', 'desc_topic_9',
       'desc_topic_10', 'desc_topic_11', 'desc_topic_12', 'desc_topic_13',
       'desc_topic_14', 'desc_topic_15', 'desc_topic_16', 'desc_topic_17',
       'desc_topic_18', 'desc_topic_19', 'desc_topic_20', 'InvoiceYear',
       'InvoiceMonth', 'InvoiceDay', 'InvoiceHour', 'Weekday'],
      dtype='object')

### Feature Scaling 

In [20]:
import numpy as np
from sklearn.preprocessing import StandardScaler

original_cols = ['Monetary', 'TotalPrice', 'Frequency', 'Recency', 'Quantity', 'UnitPrice', 'customer_lifetime_days']
scaler = StandardScaler()

# Standardize all columns at once (more efficient)
data[original_cols] = scaler.fit_transform(data[original_cols])

In [21]:
data.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,TotalPrice,Recency,Frequency,Monetary,customer_lifetime_days,...,desc_topic_16,desc_topic_17,desc_topic_18,desc_topic_19,desc_topic_20,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour,Weekday
0,541431.0,23166.0,393.586266,-0.423226,12346.0,239.734409,-0.112083,-0.455338,-0.361903,-1.919921,...,0.032051,-0.062761,0.067446,0.034499,-0.057299,2011,1,18,10,1
1,537626.0,85116.0,-0.005989,-0.180216,12347.0,0.009713,-0.112083,-0.332464,-0.220753,0.978471,...,-0.032339,0.023959,-0.022264,0.033915,0.013968,2010,12,7,14,1
2,537626.0,22375.0,-0.048423,0.312681,12347.0,-0.015763,-0.112083,-0.332464,-0.220753,0.978471,...,-0.088507,0.119232,0.051086,0.038123,-0.035789,2010,12,7,14,1
3,537626.0,71477.0,-0.005989,0.083426,12347.0,0.052589,-0.112083,-0.332464,-0.220753,0.978471,...,-0.066149,-0.013725,0.000939,0.012026,0.097659,2010,12,7,14,1
4,537626.0,22492.0,0.121313,-0.512635,12347.0,0.004121,-0.112083,-0.332464,-0.220753,0.978471,...,-0.052948,0.087200,0.028290,0.025431,-0.012222,2010,12,7,14,1


### Data Split - Spliting the data into training, validation and test sets

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# X = data.drop(columns=['Monetary'])
# y = data['Monetary']

In [24]:
# X.head()

In [25]:
# # Option 1A: Use Monetary as CLV (simplest)
# y = df['Monetary']  # Total customer spend so far

# # Option 1B: Calculate more sophisticated CLV
# df['CLV'] = df['Monetary'] * (df['Frequency'] / df['customer_lifetime_days']) * 365
# # This estimates annual customer value
# y = df['CLV']

# # Option 1C: Future CLV prediction
# # If you want to predict future value, create a forward-looking metric
# df['Average_Order_Value'] = df['Monetary'] / df['Frequency']
# df['Purchase_Rate'] = df['Frequency'] / df['customer_lifetime_days']
# df['Predicted_CLV'] = df['Average_Order_Value'] * df['Purchase_Rate'] * 365
# y = df['Predicted_CLV']

In [26]:
train, test= train_test_split(data,test_size= 0.25, random_state= 42)

In [27]:
train

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,TotalPrice,Recency,Frequency,Monetary,customer_lifetime_days,...,desc_topic_16,desc_topic_17,desc_topic_18,desc_topic_19,desc_topic_20,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour,Weekday
344847,546625.0,22621.0,-0.053727,-0.329232,17542.0,-0.055066,-0.112083,-0.439637,-0.356311,-1.919921,...,0.000000,0.000000,0.000000,0.000000,0.000000,2011,3,15,11,1
396076,542647.0,22699.0,-0.037814,0.014650,18245.0,-0.013588,-0.112083,-0.335877,-0.279782,0.835536,...,0.180334,-0.044532,0.631103,-0.226404,-0.184652,2011,1,31,11,0
129049,571653.0,22603.0,-0.005989,-0.496588,14298.0,-0.041737,-0.112083,0.662812,1.303811,0.914944,...,-0.072277,-0.020107,0.117229,0.154251,0.022933,2011,10,18,12,1
290058,537794.0,21670.0,-0.037814,-0.375083,16713.0,-0.045279,-0.112083,-0.025280,-0.137890,0.843477,...,-0.019053,-0.212512,0.114315,0.059623,-0.013548,2010,12,8,13,2
83829,549729.0,22386.0,-0.053727,-0.184802,13634.0,-0.049194,-0.112083,-0.328368,-0.310318,0.406733,...,0.096814,-0.130169,-0.085709,-0.004978,0.084529,2011,4,11,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131446,573160.0,22945.0,-0.005989,-0.466785,14359.0,-0.036890,-0.112083,-0.417110,-0.332999,-1.379947,...,-0.049529,-0.080006,-0.057602,-0.047151,0.017128,2011,10,28,8,4
283998,569148.0,21259.0,-0.059031,0.702413,16613.0,-0.031609,-0.112083,-0.436224,-0.341645,-1.919921,...,-0.032403,-0.093736,-0.044664,0.084236,0.048460,2011,9,30,15,4
144826,564304.0,22507.0,-0.053727,0.473159,14527.0,-0.022443,-0.112083,0.233437,-0.109360,1.026116,...,0.096647,-0.120394,0.151428,0.334587,0.266729,2011,8,24,12,2
161287,553176.0,48138.0,-0.064336,1.160922,14689.0,-0.043881,-0.112083,-0.447829,-0.358209,-1.919921,...,0.038334,0.042092,-0.000277,0.097849,-0.051266,2011,5,15,11,6


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272339 entries, 344847 to 133741
Data columns (total 36 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   InvoiceNo               272339 non-null  float64
 1   StockCode               272339 non-null  float64
 2   Quantity                272339 non-null  float64
 3   UnitPrice               272339 non-null  float64
 4   CustomerID              272339 non-null  float64
 5   TotalPrice              272339 non-null  float64
 6   Recency                 272339 non-null  float64
 7   Frequency               272339 non-null  float64
 8   Monetary                272339 non-null  float64
 9   customer_lifetime_days  272339 non-null  float64
 10  Country_Label           272339 non-null  int64  
 11  desc_topic_1            272339 non-null  float64
 12  desc_topic_2            272339 non-null  float64
 13  desc_topic_3            272339 non-null  float64
 14  desc_topic_4        

In [29]:
## Train dataset
train_x = train.drop(['Monetary'], axis=1).values
train_y = train[['Monetary']].values.ravel()

## Test dataset
test_x =test.drop(['Monetary'], axis=1).values
test_y =test[['Monetary']].values.ravel()

##  Splitting this train data inot train and validation

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

signature=infer_signature(train_x, train_y)

### ANN Model

In [30]:
def train_model(param, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    
    ## Define model Architecture
    mean= np.mean(train_x, axis=0)                              ##Normalization
    var=np.var(train_x,axis=0)
    
    model=keras.Sequential(
    [
        keras.Input([train_x.shape[1]]),                        ##Input shape {Number of columns}
        keras.layers.Normalization(mean=mean, variance=var),    ## Normalization
        keras.layers.Dense(64, activation ='relu'),             ## Hidden Neurons {64  layers}
        keras.layers.Dense(1)                                   ## Output Layer {1 output layer}
    ])
    
    ## Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate =param["lr"],                             ## train with differernt learning rate 
        momentum =param['momentum']                             ## train with multile momentum
    ),
                  loss="mean_squared_error",
                  metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    ## Trean the ANN model with lr and momentum parameters with MLFlow trackering
    with mlflow.start_run(nested = True):
        model.fit(train_x, train_y, validation_data= (valid_x, valid_y),
                 epochs=epochs,
                 batch_size=64
                 )
        ## Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]
        
        ## Log the parameters and results
        mlflow.log_params(param)
        mlflow.log_metric("eval_rmse", eval_rmse)
        
        ## Log the model
        mlflow.tensorflow.log_model(model, "model", signature= signature)
        
        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [31]:
def objective(params):
    ## MLFlow will track the parameters and results for each run
    result = train_model(
        params,
        epochs =3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y,
    )
    return result

In [32]:
## Set all parameters
space={
    "lr": hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}


In [33]:
mlflow.set_experiment("/customer-life-value-predicion")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials= Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )
    
    
    # featch the details of the best run
    best_run = sorted(trials.results, key=lambda x:x["loss"])[0]
    
    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature= signature)
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    
    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")
    

Epoch 1/3                                            

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2025-06-09 06:57:26.856701: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


   1/3405 ━━━━━━━━━━━━━━━━━━━━ 27:59 493ms/step - loss: 0.6805 - root_mean_squared_error: 0.8249
  24/3405 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.9541 - root_mean_squared_error: 0.9749     
  50/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.0289 - root_mean_squared_error: 1.0127
  76/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.0878 - root_mean_squared_error: 1.0411
  92/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.1073 - root_mean_squared_error: 1.0506
 119/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.1421 - root_mean_squared_error: 1.0669
 136/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.1659 - root_mean_squared_error: 1.0779
 162/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.1976 - root_mean_squared_error: 1.0923
 187/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.2163 - root_mean_squared_error: 1.1009
 210/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.2274 - root_mean_squared_error: 1.1060
 234/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.2328 - root_mean_

2025/06/09 06:58:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpcuz3njeg/model, flavor: tensorflow). Fall back to return ['tensorflow==2.19.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



Epoch 1/3                                                                      

   1/3405 ━━━━━━━━━━━━━━━━━━━━ 24:16 428ms/step - loss: 1.0531 - root_mean_squared_error: 1.0262
  13/3405 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 1.3599 - root_mean_squared_error: 1.1648    
  34/3405 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 1.5625 - root_mean_squared_error: 1.2473
  43/3405 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 1.6041 - root_mean_squared_error: 1.2640
  47/3405 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 1.6144 - root_mean_squared_error: 1.2682
  70/3405 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 1.6423 - root_mean_squared_error: 1.2798
  93/3405 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 1.6527 - root_mean_squared_error: 1.2843
 115/3405 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 1.6629 - root_mean_squared_error: 1.2884
 142/3405 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 1.6699 - root_mean_squared_error: 1.2913 
 151/3405 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 1.6701 - root_mean_squ

...

Epoch 1/3                                                                      

   1/3405 ━━━━━━━━━━━━━━━━━━━━ 24:48 437ms/step - loss: 1.1983 - root_mean_squared_error: 1.0947
  15/3405 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 1.4041 - root_mean_squared_error: 1.1839    
  38/3405 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 1.6247 - root_mean_squared_error: 1.2721 
  60/3405 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6286 - root_mean_squared_error: 1.2745
  82/3405 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 1.6042 - root_mean_squared_error: 1.2652
 110/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.5636 - root_mean_squared_error: 1.2491
 126/3405 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 1.5422 - root_mean_squared_error: 1.2405
 146/3405 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 1.5181 - root_mean_squared_error: 1.2307
 171/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.4924 - root_mean_squared_error: 1.2201
 192/3405 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.4724 - root_mean_squared_er

2025/06/09 07:00:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp8sdarva6/model, flavor: tensorflow). Fall back to return ['tensorflow==2.19.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



100%|██████████| 4/4 [02:45<00:00, 41.44s/trial, best loss: 0.8176860213279724]


2025/06/09 07:00:20 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpcjk6o9m6/model, flavor: tensorflow). Fall back to return ['tensorflow==2.19.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 


Best parameters: {'lr': np.float64(0.0005853766459010583), 'momentum': np.float64(0.15732608720289332)}
Best eval rmse: 0.8176860213279724
🏃 View run adventurous-elk-260 at: http://127.0.0.1:5000/#/experiments/523884947611096273/runs/7a06614c24f04d239c691b97c018e07f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/523884947611096273
